# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [1]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, show_map
import math
from queue import PriorityQueue

%load_ext autoreload
%autoreload 2

### Map Basics

In [2]:
map_10 = load_map_10()
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [4]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [5]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [6]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [7]:
map_40.roads


[[36, 34, 31, 28, 17],
 [35, 31, 27, 26, 25, 20, 18, 17, 15, 6],
 [39, 36, 21, 19, 9, 7, 4],
 [35, 20, 15, 11, 6],
 [2, 39, 36, 21, 19, 9, 7],
 [32, 16, 14],
 [1, 3, 35, 20, 15, 11],
 [2, 4, 39, 36, 22, 21, 19, 9],
 [33, 30, 14],
 [2, 4, 7, 36, 21, 19],
 [31, 27, 26, 25, 24, 18, 17, 13],
 [3, 6, 35, 20, 15],
 [37, 34, 31, 28, 22, 17],
 [10, 27, 24, 18],
 [5, 8, 33, 30, 16],
 [1, 3, 6, 11, 35, 31, 26, 25, 20, 17],
 [5, 14, 37, 30],
 [0, 1, 10, 12, 15, 34, 31, 28, 26, 25, 18],
 [1, 10, 13, 17, 31, 27, 26, 25, 24],
 [2, 4, 7, 9, 21],
 [1, 3, 6, 11, 15, 35, 26],
 [2, 4, 7, 9, 19],
 [7, 12, 39, 37, 29],
 [38, 32, 29],
 [10, 13, 18, 27],
 [1, 10, 15, 17, 18, 34, 31, 27, 26],
 [1, 10, 15, 17, 18, 20, 25, 34, 31, 27],
 [1, 10, 13, 18, 24, 25, 26, 31],
 [0, 12, 17, 39, 36, 34, 31],
 [22, 23, 38, 37, 32],
 [8, 14, 16, 33],
 [0, 1, 10, 12, 15, 17, 18, 25, 26, 27, 28, 34],
 [5, 23, 29, 38],
 [8, 14, 30],
 [0, 12, 17, 25, 26, 28, 31],
 [1, 3, 6, 11, 15, 20],
 [0, 2, 4, 7, 9, 28, 39],
 [12, 16, 22, 

In [8]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

## The Algorithm
### Writing your algorithm
The algorithm written will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`. However you must complete several methods before it will work.

```bash
> PathPlanner(map_40, 5, 34).path
[5, 16, 37, 12, 34]
```

In [9]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search()
            return self.path
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            current = self.get_current_node()
            

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                self.path.pop(0)
                return self.path
            else:
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet.queue:
                    # Discover a new node
                    if self.get_tenative_gScore(current,neighbor) < self.get_gScore(neighbor):
                        self.record_best_path_to(current, neighbor)
                        self.openSet.put((self.fScore[neighbor],neighbor))
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
               
        print("No Path Found")
        self.path = None
        return False

Create the following methods:

In [10]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # TODO: return a data structure suitable to hold the set of nodes already evaluated
    return set()

In [11]:
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        p=PriorityQueue()
        p.put((0,self.start))
        return p
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")
    

In [12]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    return {self.start:None}

In [13]:
from collections import defaultdict
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO:  a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    gScore=defaultdict(lambda: float(9999999999))
    gScore[self.start]=0
    return gScore

In [14]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    fScore={}
    fScore[self.start]= self.heuristic_cost_estimate(self.start)
    return fScore
        


In [15]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    # TODO: Set map to new value. 
    self.map=M

In [16]:
def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    self.start=start
    self.goal=None
    self.closedSet=None
    self.openSet=None
    self.cameFrom=None
    self.gScore=None
    self.fScore=None
    self.path=None
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    

In [17]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    self.goal=goal


In [18]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    return self.openSet.get()[1]


In [19]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    return self.map.roads[node]


In [20]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    return self.gScore[node]


In [21]:
def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the g Score of the current node 
    # plus distance from the current node to it's neighbors
    return self.get_gScore(current) + self.distance(self.map.intersections[current],self.map.intersections[neighbor])

In [22]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the open set is empty. False otherwise.
    return self.openSet.empty()

In [23]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distanc
    distance=math.sqrt((((node_1[0]-node_2[0])**2)+(node_1[1]-node_2[1])**2))
    return distance
    

In [24]:
def heuristic_cost_estimate(self,node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    return self.distance(self.map.intersections[node],self.map.intersections[self.goal])

In [25]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    return self.get_gScore(node)+ self.heuristic_cost_estimate(node)
    

In [26]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor]=current
    self.gScore[neighbor]=self.get_tenative_gScore(current,neighbor)
    self.fScore[neighbor]=self.calculate_fscore(neighbor)
    


In [27]:

PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to
planner = PathPlanner(map_40, 5, 34)
PathPlanner._reset = planner._reset

In [28]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [29]:
from test import test

test(PathPlanner)

All tests pass! Congratulations!


## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **: A* Algorithm uses the property of PriorityQueue to keep the element with lowest fScore between all the current,neighbor pairs at the top, which we can then take out and add to our path. It uses a heuristic which is not overestimating and is Admissible(value lower than the actual value).

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **: A uniform cost search is unidirectional, going in every direction till it gets to the goal node with low score. It wastes alot of processing cycles, To fix this we provided a direction with heuristic function, a heuristic function which is admissible helps direct the search in the direction of goal node. It diverts the searching algorithm to move to goal node and help reduce the processing cycles. Best First search are algorithms which are non-greedy based. They uses a heuristic to overcome a bruteforce technique. Examples: A*, B*

- What is a heuristic?

** ANSWER **: 
 A heuristic is a function which helps in determining if the current node is closer to Goal node or not. thus, it provides a direction to the searching algorithm.

- OpenSet= Frontier neighbors

- ClosedSet=Explored neighbors

- heuristic=euclidean distance between (neighbor,goal)

- g_score= distance between (current,neighbor)

- f_score=g_score+heuristic 



- What is a consistent heuristic?

** ANSWER **: A heurisitic function is consistent when the estimated cost from current to goal node is more than cost from (current,neighbor) + cost from (neighbor,goal) 
I.e ` C(current,neighbor) + C(neighbor,goal) <= C(current,goal)`

- What is a admissible heuristic? 

** ANSWER **: A heuristic which underestimate the state values is considered as admissible heuristic. 
Eg Euclidean distance between neighbor node and goal node is always less than or equal to the true cost from neighbor to goal node.

- ___ admissible heuristic are consistent.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **:  Some

- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: All